In [ ]:
%reset -f
%load_ext autoreload
%autoreload 2

import mechanics
from mechanics import *
from mechanics.lagrange import euler_lagrange_equation
import mechanics.space as space


t, = base_spaces('t')
def dot(f): return diff(f, t)

r, = variables('r', t)
theta, = variables(r'\theta', t, space=space.S)
q = r, theta
dq = tuple(dot(q_n) for q_n in q)
ddq = tuple(dot(dq_n) for dq_n in dq)

mu, m = constants(r'\mu, m')

U = - mu / r
T = Rational(1, 2) * m * (dot(r)**2 + (r * dot(theta))**2)
E = T + U
L = T - U
show('L =' , L)

EL = euler_lagrange_equation(L, q)
show_equation(EL)

F = solve(EL, ddq)
show_equation(F)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
diff(L, dot(r))

m()/2